In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg
!pip install nbstripout
!pip install OpenCC

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
4

In [9]:
from opencc import OpenCC
import whisper
import json

In [6]:
folder_path = '/content/drive/My Drive/Ephesians_Book/'
audio_file_prefix = 'part'
text_file_prefix = audio_file_prefix
cc = OpenCC('t2s')
# Load the model (you can use "small", "medium", or "large" — large = best accuracy)
model = whisper.load_model("small")  # medium is good for Chinese & faster than large

100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 48.9MiB/s]


In [8]:
def file_name_builder(folder_path, file_prefix, file_extension, index):
  return f"{folder_path}{file_prefix}_{index:03d}.{file_extension}"

In [10]:
def convert_audio_files_to_text_json(start_index, end_index):
  for i in range(start_index, end_index):
    audio_file_name = file_name_builder(f"{folder_path}audio_files_experiment/",
                                        audio_file_prefix, "m4a", i)
    print(f"processing file {audio_file_name}")

    result = model.transcribe(audio_file_name, language="zh")
    # convert any traditional Chinese to simplified Chinese
    segment_object_list = transcription_result_to_objects(result)

    json_file_name = file_name_builder(f"{folder_path}text_files_experiment/",
                                      text_file_prefix, "json", i)
    with open(json_file_name, "w", encoding="utf-8") as f:
      json.dump(segment_object_list, f, ensure_ascii=False, indent=2)

In [11]:
def transcription_result_to_objects(result):
  segment_object_list = []
  for segment in result["segments"]:
    segment_object = {
        "content": cc.convert(segment["text"]),
        "startTime": segment["start"],
        "endTime": segment["end"],
        "titleIndex": None
    }
    segment_object_list.append(segment_object)
  return segment_object_list

In [13]:
convert_audio_files_to_text_json(0, 3)

processing file /content/drive/My Drive/Ephesians_Book/audio_files_experiment/part_000.m4a
processing file /content/drive/My Drive/Ephesians_Book/audio_files_experiment/part_001.m4a
processing file /content/drive/My Drive/Ephesians_Book/audio_files_experiment/part_002.m4a
